In [1]:
import requests
import time
from bs4 import BeautifulSoup 
from selenium import webdriver
import re
from db_context import MongoDb
headers = {"User-Agent": 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36 OPR/99.0.0.0'}
#&index [1-7]
scratch_list_url = 'https://www.lotto.pl/api/lotteries/zdrapki?name=&status=&yearSalesEnd=&page=70&scratchStatuses=Standard,Novelity,LastChance,Announcement,SalesEnd,Debut&index=1&size=12&sort=DateSalesUtc&order=DESC'

In [ ]:
#read info about newest scratch cards
response = requests.get(scratch_list_url, headers=headers)
print(response.status_code, response.reason)
scratch_cards_data = response.json()

In [ ]:
#save result to avoid creating more requests
import json
with open('scratch_card.json', 'w') as outfile:
    json.dump(scratch_cards_data, outfile)

In [7]:
import json
with open('scratch_card.json') as json_file:
    scratch_cards_data = json.load(json_file)
    print(type(scratch_cards_data), scratch_cards_data)

<class 'dict'> {'totalRows': 79, 'items': [{'result': {'displayContent': True, 'highestWin': None, 'popularity': 175, 'status': 'Announcement', 'number': 689, 'drawPrice': 2, 'pictureUrl': 'storage/files/2023/7/18/105c82ff-3082-41b5-8b40-92ce4c276a07/news_600x340.png', 'name': 'MIĘTOWA KASA', 'id': 1323, 'hit': False, 'dateSales': '2023-08-02T22:00:00Z', 'dateSalesEnd': None}, 'meta': {'url': '/zdrapki/katalog-zdrapek/mietowa-kasa,1323'}, 'code': 200}, {'result': {'displayContent': True, 'highestWin': 200000, 'popularity': 649, 'status': 'Debut', 'number': 687, 'drawPrice': 5, 'pictureUrl': 'storage/files/2023/7/5/4b009cb6-e8da-4be2-b5da-0d48ae7456be/news_600x340.png', 'name': 'WALIZKA PIENIĘDZY', 'id': 1322, 'hit': False, 'dateSales': '2023-07-19T22:00:00Z', 'dateSalesEnd': None}, 'meta': {'url': '/zdrapki/katalog-zdrapek/walizka-pieniedzy,1322'}, 'code': 200}, {'result': {'displayContent': True, 'highestWin': 500000, 'popularity': 760, 'status': 'Novelity', 'number': 686, 'drawPrice'

In [9]:
def get_scratch_card_info(scratch_card_name, scratch_card_url):
    

    #read info about single scratch_card
    driver = webdriver.Chrome()
    driver.get('https://www.lotto.pl'+scratch_card_url)
    content = driver.page_source
    time.sleep(1)
    driver.close()
    soup = BeautifulSoup(content)
    res = soup.find(string=re.compile('wygrane:')).get_text().replace('\xa0', '')
    pot, circulation = re.findall('(\d+)', res)

    scratch_card = {'name': scratch_card_name,
                    'url': scratch_card_url,
                    'pot': pot,
                    'circulation': circulation,
                    'win_values': [],
                    'n_wins': [],
                    'wins_left': []
                    }

    table = soup.select('table td')

    scratch_card['win_values'] = [ table[i].get_text().replace('\xa0', '').replace('zł', '').strip() for i in range(3, len(table),3)]
    scratch_card['n_wins'] = [table[i].get_text() for i in range(4, len(table),3)]
    scratch_card['wins_left'] = [table[i].get_text() for i in range(5, len(table),3)]
    return scratch_card

In [10]:
name = scratch_cards_data['items'][0]['result']['name']
url = scratch_cards_data['items'][0]['meta']['url']
print(name, url)
result = get_scratch_card_info(name, url)

MIĘTOWA KASA /zdrapki/katalog-zdrapek/mietowa-kasa,1323


In [11]:
db = MongoDb('mongodb://localhost:27017')

print(result)
print(db.insert_one(result))

{'name': 'MIĘTOWA KASA', 'url': '/zdrapki/katalog-zdrapek/mietowa-kasa,1323', 'pot': '22438500', 'circulation': '21000000', 'win_values': ['44000', '1000', '100', '50', '20', '10', '5', '2'], 'n_wins': ['14', '210', '5600', '42000', '315000', '210000', '577500', '3832500'], 'wins_left': ['14', '210', '5600', '42000', '315000', '210000', '577500', '3832500']}
64ba7fe6a3de908d2d713d03


In [12]:
for item in scratch_cards_data['items']:
    name = item['result']['name']
    url = item['meta']['url']
    result = get_scratch_card_info(name, url)
    db.insert_one(result)
